# CNN Tensor Creation

In [1]:
import importlib

# List of libraries to check
libraries = [
    'numpy',
    'matplotlib',
    'scipy',
    'torch',
    'torchdiffeq',
    'torchsummary',
    'minepy',
    'pyrqa',
    'pyts',
    'MFDFA',
    'pyinform',
    'graphviz',
    'fa2',
    'networkx'
]

for lib in libraries:
    try:
        module = importlib.import_module(lib)
        version = getattr(module, '__version__', 'Unknown version')
        print(f'{lib}: {version}')
    except ImportError:
        print(f'{lib} is not installed.')

numpy: 1.24.3
matplotlib: 3.7.0
scipy: 1.10.1
torch: 2.0.1+cu117
torchdiffeq: 0.2.3
torchsummary: Unknown version
minepy: b'1.2.6'
pyrqa: Unknown version
pyts: 0.12.0
MFDFA: 0.4.3
pyinform: Unknown version
graphviz: 0.20.1
fa2 is not installed.
networkx: 3.0


# EEG

In [3]:
import torch
from sklearn.preprocessing import StandardScaler
import numpy as np

# Load EEG data
EEG_data = np.load('/home/vincent/AAA_projects/MVCS/Neuroscience/eeg_data_with_channels.npy', allow_pickle=True)

# Standardize the EEG data
scaler = StandardScaler()
EEG_data_standardized = scaler.fit_transform(EEG_data)

# Convert the data to a PyTorch tensor
eeg_tensor = torch.tensor(EEG_data_standardized, dtype=torch.float32)

eeg_tensor = eeg_tensor.T  # Transpose to make channels the second dimension
eeg_tensor = eeg_tensor.unsqueeze(0).unsqueeze(2)  # Add batch and height dimensions
# Now eeg_tensor should have shape [1, num_channels, 1, num_timepoints]

print(f"The reshaped tensor has shape: {eeg_tensor.shape}")

# Step 4: Save the tensor
save_path = "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/EEG_tensor.pth"
torch.save(eeg_tensor, save_path)


The reshaped tensor has shape: torch.Size([1, 32, 1, 4227788])


# Band Powers

In [9]:
import numpy as np
import torch

# Load the band powers data from the .npy file
band_powers_data = np.load('/home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/Spectral Analysis/BandPowers_x.npy', allow_pickle=True).item()

# Print the populated keys in band_powers_data (channels)
print("Populated keys in band_powers_data:", list(band_powers_data.keys()))

# Extract eeg_channels from the data (assuming they're the top-level keys)
eeg_channels = list(band_powers_data.keys())

# Extract frequency_bands from the data (assuming they're the second-level keys for any channel)
frequency_bands = list(next(iter(band_powers_data.values())).keys())

# Assuming band_powers_data is in the format {channel: {band: values}}
# Convert the band powers dictionary to a tensor
num_time_steps = len(next(iter(next(iter(band_powers_data.values())).values())))
tensor_shape = (num_time_steps, len(eeg_channels), len(frequency_bands))
band_power_tensor = torch.empty(tensor_shape)

for i, channel in enumerate(eeg_channels):
    for j, band in enumerate(frequency_bands):
        band_power_tensor[:, i, j] = torch.tensor(band_powers_data[channel][band])

# Saving path for the band power tensor
save_path = "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/band_power_tensor.pth"

# Save the tensor
torch.save(band_power_tensor, save_path)


Populated keys in band_powers_data: ['Fp1', 'Fpz', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 'M1', 'T7', 'C3', 'Cz', 'C4', 'T8', 'M2', 'CP5', 'CP1', 'CP2', 'CP6', 'P7', 'P3', 'Pz', 'P4', 'P8', 'POz', 'O1', 'Oz', 'O2']


# Fast Fourier Transform

In [14]:
import numpy as np
import scipy.signal
import torch

# Load the FFT PSD data
load_path = '/home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/Spectral Analysis/combined_fft_psd_x.npy'
combined_fft_psd_data = np.load(load_path, allow_pickle=True).item()

eeg_channels = ['Fp1', 'Fpz', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6',
                'M1', 'T7', 'C3', 'Cz', 'C4', 'T8', 'M2', 'CP5', 'CP1', 'CP2', 'CP6',
                'P7', 'P3', 'Pz', 'P4', 'P8', 'POz', 'O1', 'Oz', 'O2']

num_channels = len(combined_fft_psd_data.keys())
# Assuming all PSD sequences have the same length
num_frequencies = len(next(iter(combined_fft_psd_data.values())))
# Reshape the data
fft_psd_tensor = torch.empty((num_channels, num_frequencies))

for i, channel in enumerate(eeg_channels):
    fft_psd_tensor[i] = torch.tensor(combined_fft_psd_data[channel])
    
# Saving path for the CNN tensor
save_path = "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/fast_fourier_transform_psd_tensor.pth"

# Save the tensor
torch.save(fft_psd_tensor, save_path)

# Short Time Fourier Transform

In [17]:
import numpy as np
import torch

# Load the STFT data
stft_data_dict = np.load('/home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/Spectral Analysis/STFT_x.npy', allow_pickle=True).item()

# Get number of channels, frequencies, and time intervals from the data
num_channels = len(stft_data_dict)
num_frequencies, num_time_intervals = stft_data_dict[next(iter(stft_data_dict.keys()))].shape

# Initialize a tensor to store the STFT data
stft_tensor = torch.empty((num_channels, num_frequencies, num_time_intervals))

# Populate the tensor
for i, channel_data in enumerate(stft_data_dict.values()):
    stft_tensor[i] = torch.tensor(channel_data)


# Saving path for the CNN tensor
save_path = "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/short_time_fourier_transform_tensor.pth"

# Save the tensor
torch.save(stft_tensor, save_path)

# 3D Embedding data EEG

In [26]:
import numpy as np
import zipfile
import os

eeg_channels = ['Fp1', 'Fpz', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6',
                'M1', 'T7', 'C3', 'Cz', 'C4', 'T8', 'M2', 'CP5', 'CP1', 'CP2', 'CP6',
                'P7', 'P3', 'Pz', 'P4', 'P8', 'POz', 'O1', 'Oz', 'O2']

# Extract zip file
zip_path = '/home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/Phase Space/3dembedded_data.zip'
extract_path = '/home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/Phase Space/3dembedding_data/temp'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Load the first channel to determine the number of data points
first_channel_data = np.load(os.path.join(extract_path, f'3dembedded_{eeg_channels[0]}.npy'))
num_data_points, emb_dim = first_channel_data.shape
num_channels = len(eeg_channels)

# Initialize a 4D tensor to store embeddings
eeg_tensor = np.zeros((num_data_points, emb_dim, num_channels, 1))

# Populate the tensor
for idx, channel in enumerate(eeg_channels):
    eeg_tensor[:, :, idx, 0] = np.load(os.path.join(extract_path, f'3dembedded_{channel}.npy'))

# Saving path for the CNN tensor
save_path = "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/3D_embedding_EEG_tensor.pth"

# Save the tensor
torch.save(eeg_tensor, save_path)

ValueError: could not broadcast input array from shape (4227590,3) into shape (4227588,3)

# Hurst Exponents

In [69]:
import numpy as np
import torch

# Load Data
hurst_exponents = np.load('/home/vincent/AAA_projects/MVCS/Neuroscience/HurstExponents/hurst_exponents.npy')

# Reshape and normalize data
hurst_exponents = hurst_exponents.reshape(-1, 1)  # Reshape to (num_channels, 1)
normalized_data = (hurst_exponents - np.mean(hurst_exponents)) / np.std(hurst_exponents)

# Convert to PyTorch tensor and reshape for CNN
hurst_tensor = torch.tensor(normalized_data).float().unsqueeze(0).unsqueeze(1)  # Shape: (1, 1, num_channels, 1)

print(hurst_tensor.shape)  # should print torch.Size([1, 1, 32, 1])

# Saving path for the CNN tensor
save_path = "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/Hurst_tensor.pth"

# Save the tensor
torch.save(hurst_tensor, save_path)

torch.Size([1, 1, 32, 1])


# MFDFA

In [68]:
import numpy as np
import torch

# Load the saved numpy features
cnn_features = np.load('/home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/MFDFA/cnn_mfdfa.npy')

# Reshape the Data to include a channel dimension
cnn_features_tensor = cnn_features[..., np.newaxis]  # Adding a channel dimension

# Convert to PyTorch tensor
cnn_features_tensor_torch = torch.tensor(cnn_features_tensor, dtype=torch.float32)

print(cnn_features_tensor_torch.shape)  

# Save the tensor using torch.save
save_path = "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/mfdfa_tensor.pth"
torch.save(cnn_features_tensor_torch, save_path)

torch.Size([9, 32, 10, 1])


# MFDFA Concatenated

In [67]:
import torch
import numpy as np

# Load the MFDFA results from the file
save_dir = '/home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/MFDFA/'  
mfdfa_results_np = np.load(save_dir + 'MFDFA_results.npy', allow_pickle=True)

# Extract the dictionary from the numpy scalar
mfdfa_results = mfdfa_results_np.item()

# Extract the channels data from your dictionary
channels_data = [mfdfa_results[key] for key in sorted(mfdfa_results.keys())]

# List to store the data for each channel
channel_images = []

for channel_data in channels_data:
    scales, fluctuations = channel_data
    # Reshape the scales and fluctuations into column vectors
    scales = scales.reshape(-1, 1)
    fluctuations = fluctuations.mean(axis=1).reshape(-1, 1)  # Take the mean of fluctuations across its scales for simplicity
    
    # Concatenate the scales and fluctuations horizontally to form an "image"
    channel_image = np.hstack([scales, fluctuations])
    channel_images.append(channel_image)

# Convert the list of channel images to a tensor
tensor = torch.from_numpy(np.array(channel_images)).float()

# Reshape the tensor to [batch_size, channels, height, width]
mfdfa_concatd_tensor = tensor.unsqueeze(1)  # Adding a channel dimension

print(mfdfa_concatd_tensor.shape) 

# Save the tensor using torch.save
save_path = "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/mfdfa_concatd_tensor.pth"
torch.save(mfdfa_concatd_tensor, save_path)

torch.Size([32, 1, 30, 2])


# Arnold Tongues Rotation Numbers

In [66]:
import numpy as np
import torch

# Load the saved rotation numbers
rotation_numbers_path = '/home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/Arnold Tongues/rotation_numbers.npy'
rotation_numbers_dict = np.load(rotation_numbers_path, allow_pickle=True).item()

# EEG channels definition (as per your previous code)
eeg_channels = ['Fp1', 'Fpz', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6',
                'M1', 'T7', 'C3', 'Cz', 'C4', 'T8', 'M2', 'CP5', 'CP1', 'CP2', 'CP6',
                'P7', 'P3', 'Pz', 'P4', 'P8', 'POz', 'O1', 'Oz', 'O2']

# Omegas and K_values are derived from your earlier code
omegas = np.linspace(0, 1, 300)
K_values = np.linspace(0, 2 * np.pi, 300)

tensor_shape = (len(eeg_channels), len(K_values), len(omegas))
rotation_numbers_tensor = np.zeros(tensor_shape)

for idx, ch in enumerate(eeg_channels):
    rotation_numbers_tensor[idx] = rotation_numbers_dict[ch]

# Convert the numpy tensor to a PyTorch tensor
rotation_numbers_torch_tensor = torch.tensor(rotation_numbers_tensor)

print(rotation_numbers_torch_tensor.shape)  

# Save the PyTorch tensor to disk
torch_save_path = '/home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/Arnold Tongues/rotation_numbers_tensor.pt'
torch.save(rotation_numbers_torch_tensor, torch_save_path)

torch.Size([32, 300, 300])


# Transfer Entropy Hemispheric

In [65]:
import torch
import numpy as np

# Load the saved CNN input data
cnn_input_path = '/home/vincent/AAA_projects/MVCS/Neuroscience/Features/CNN/cnn_transfer_entropy_hemispheric_avg_input.npy'
cnn_input_data = np.load(cnn_input_path)

# Convert the numpy array to a PyTorch tensor
cnn_input_tensor = torch.tensor(cnn_input_data)

print(cnn_input_tensor.shape)  

# Save the PyTorch tensor to disk
torch_save_path = '/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/transfer_entropy_hemispheric_avg_input_tensor.pt'
torch.save(cnn_input_tensor, torch_save_path)

torch.Size([92, 92])


# Transfer Entropy Regional

In [64]:
import numpy as np

# Load the saved CNN input data
cnn_input_path = '/home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/Transfer Entropy/regional_transfer_entropy_results.npy'
cnn_input_data = np.load(cnn_input_path, allow_pickle=True).item()

regions = ["Frontal", "Temporal", "Parietal", "Occipital"]
TE_matrix = np.zeros((4, 4))

for i, source in enumerate(regions):
    for j, target in enumerate(regions):
        if i != j:
            key = f"{source}_to_{target}"
            TE_matrix[i, j] = cnn_input_data[key]

print(TE_matrix)

# Convert the numpy array to a PyTorch tensor
cnn_input_tensor = torch.tensor(TE_matrix)

print(cnn_input_tensor.shape)  

# Save the PyTorch tensor to disk
torch_save_path = '/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/transfer_entropy_regional_tensor.pt'
torch.save(cnn_input_tensor, torch_save_path)

[[0.         0.25110662 0.57730508 0.67410323]
 [0.33752969 0.         0.47118759 0.61260954]
 [0.38813154 0.22015982 0.         0.63537753]
 [0.42216949 0.28119128 0.58496923 0.        ]]
torch.Size([4, 4])


# Transfer Entropy Granular

In [ ]:
import numpy as np
import torch

# Load the numpy results file
results_file_path = '/home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/Transfer Entropy/full_granularity_transfer_entropy_results.npy'
TE_results = np.load(results_file_path, allow_pickle=True).item()

# Create a 4x4 matrix with zeros
TE_matrix = np.zeros((32, 32))

# Fill the matrix using the provided results
for i, source_region in enumerate(regions):
    for j, target_region in enumerate(regions):
        if i != j:
            key = f"{source_region}_to_{target_region}"
            TE_matrix[i, j] = TE_results[key]

# Convert the numpy matrix to PyTorch tensor
TE_tensor = torch.tensor(TE_matrix)

print(TE_tensor.shape)  

# Save the PyTorch tensor to disk
torch_save_path = '/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/transfer_entropy_granular_tensor.pt'
torch.save(TE_tensor, torch_save_path)


# DSPM

In [77]:
import numpy as np
import torch

# Load the numpy data
cnn_data_np = np.load('/home/vincent/AAA_projects/MVCS/Neuroscience/Features/CNN/cnn_dspm.npy')

# Convert the numpy array to a PyTorch tensor
cnn_data_tensor = torch.tensor(cnn_data_np)

print(cnn_data_tensor.shape)  

# Save the tensor using PyTorch's save function
torch.save(cnn_data_tensor, '/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/dspm_tensor.pt')


torch.Size([19, 18840, 10])


# Higuchi Fractal Dimension

In [60]:
import torch
import numpy as np

# Load the CNN feature
cnn_features_path = '/home/vincent/AAA_projects/MVCS/Neuroscience/Features/CNN/cnn_fractal.npy'
cnn_features = np.load(cnn_features_path)

# Reshape for CNN (batch_size, channels, height, width)
cnn_tensor = torch.tensor(cnn_features).unsqueeze(0).unsqueeze(0)

print(cnn_tensor.shape)  

# Save the PyTorch tensor to disk
torch_save_path = '/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/higuchi_fractal_dimensions_tensor.pt'
torch.save(cnn_tensor, torch_save_path)

torch.Size([1, 1, 4, 8])


# Spectral Entropy

In [59]:
import numpy as np
import torch

# 1. Load the spectral entropy values
results_folder_path = '/home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/Spectral Analysis/'
results_file = "SpectralEntropy_x.npy"
spectral_entropy_dict = np.load(results_folder_path + results_file, allow_pickle=True).item()

# Extract only the values from the dictionary
entropy_values = list(spectral_entropy_dict.values())

# 2. Convert the spectral entropy values into a tensor
entropy_tensor = torch.Tensor(entropy_values)

# 3. Reshape the tensor for CNN (1 sample, 1 channel, 32 height (EEG channels), 1 width (spectral entropy))
cnn_tensor = entropy_tensor.view(1, 1, 32, 1)

print(cnn_tensor.shape)  # should print torch.Size([1, 1, 32, 1])

# Save the PyTorch tensor to disk
torch_save_path = '/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/spectral_entropy_tensor.pt'
torch.save(cnn_tensor, torch_save_path)

torch.Size([1, 1, 32, 1])


# Spectral Centroids

In [58]:
import numpy as np
import torch

# 1. Load the spectral centroid values
results_path = '/home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/Spectral Analysis/'
results_file = "SpectralCentroids_x.npy"
spectral_centroids_dict = np.load(results_path + results_file, allow_pickle=True).item()

# Extract only the values from the dictionary
centroid_values = list(spectral_centroids_dict.values())

# 2. Convert the spectral centroid values into a tensor
centroid_tensor = torch.Tensor(centroid_values)

# 3. Reshape the tensor for CNN (1 sample, 1 channel, 32 height (EEG channels), 1 width (spectral centroid))
cnn_tensor = centroid_tensor.view(1, 1, 32, 1)

print(cnn_tensor.shape)  # should print torch.Size([1, 1, 32, 1])

# Save the PyTorch tensor to disk
torch_save_path = '/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/spectral_centroids_tensor.pt'
torch.save(cnn_tensor, torch_save_path)

torch.Size([1, 1, 32, 1])


# Frequency of Max Power

In [71]:
import numpy as np
import torch

# Load the peak frequency data
results_path = '/home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/Spectral Analysis/'
results_file = "PeakFrequencies_x.npy"
peak_frequencies_dict = np.load(results_path + results_file, allow_pickle=True).item()

# Extract only the values from the dictionary
peak_frequency_values = list(peak_frequencies_dict.values())

# Convert the peak frequency values into a tensor
peak_frequency_tensor = torch.Tensor(peak_frequency_values)

# Reshape the tensor for CNN (1 sample, 1 channel, 32 height (EEG channels), 1 width (peak frequency))
cnn_tensor = peak_frequency_tensor.view(1, 1, 32, 1)

print(cnn_tensor.shape)  # should print torch.Size([1, 1, 32, 1])

# Save the PyTorch tensor to disk
torch_save_path = '/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/freq_max_power_tensor.pt'
torch.save(cnn_tensor, torch_save_path)

torch.Size([1, 1, 32, 1])


# Spectral Edge Frequencies

In [72]:
import numpy as np
import torch

# Load the spectral edge density data
results_path = '/home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/Spectral Analysis/'
results_file = "SpectralEdgeDensities_x.npy"
spectral_edge_densities_dict = np.load(results_path + results_file, allow_pickle=True).item()

# Extract only the values from the dictionary
spectral_edge_density_values = list(spectral_edge_densities_dict.values())

# Convert the spectral edge density values into a tensor
spectral_edge_density_tensor = torch.Tensor(spectral_edge_density_values)

# Reshape the tensor for CNN (1 sample, 1 channel, 32 height (EEG channels), 1 width (spectral edge density))
cnn_tensor = spectral_edge_density_tensor.view(1, 1, 32, 1)

print(cnn_tensor.shape)  # should print torch.Size([1, 1, 32, 1])

# Save the PyTorch tensor to disk
torch_save_path = '/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/spectral_edge_freqs_tensor.pt'
torch.save(cnn_tensor, torch_save_path)

torch.Size([1, 1, 32, 1])
